In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import sys
from datetime import datetime

from neuralnet import DNN_pred, Abs_LV, Rel_LV, Compo_LV
from utils.evaluations import calc_nondiag_score
from sklearn.metrics import average_precision_score, roc_auc_score
from utils.simulation import simulate_glv

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
abundance = np.load(r'D:\microbial_network\microbial_network_explore\data\simulated\n20_k5_random_random_1.0_1000\y.npy')
abundance = abundance / abundance.sum(axis=1, keepdims=True)
adj = np.load(r'D:\microbial_network\microbial_network_explore\data\simulated\n20_k5_random_random_1.0_1000\adj.npy')
adj_norm = adj / adj.sum(axis=1, keepdims=True)
M = np.load(r'D:\microbial_network\microbial_network_explore\data\simulated\n20_k5_random_random_1.0_1000\M.npy')

In [51]:
z, x, y, adj, M = simulate_glv(
        num_taxa=10,
        # avg_degree=np.random.randint(5, 20),
        avg_degree=2,
        time_points=10000,
        time_step=0.01,
        downsample=1,
        noise_var=0,
        max_interaction_strength=1,
    )

Node 'mgx' initialized
Interaction 'mgx->mgx' added
set m:(mgx)->(mgx):   0:10    0:10
Added x0 vector to node mgx
Added growth rates to node mgx
Initialized
Interaction 'mgx_mgx' added


In [52]:
model = DNN_pred(20, 512).to('cuda')

loss_fn = torch.nn.MSELoss().to('cuda')
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 10000, gamma=0.9)

In [53]:
epoch_num = 300000
for epoch in range(epoch_num):
    input_data = torch.from_numpy(abundance[:-1, :]).float().to('cuda')
    target_data = torch.from_numpy(abundance[1:, :]).float().to('cuda')
    # print(input_data.shape, target_data.shape)
    rel = model(input_data)
    loss = loss_fn(rel, target_data)

    # identity = torch.eye(output.shape[1]).to('cuda')
    # id_out = model(identity)
    # lamda = 0.001
    # loss = loss + lamda * torch.abs(id_out).sum()
    
    print(f"Epoch: {epoch}, Loss: {loss.item()}", end='\x1b[1K\r')
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()

KeyboardInterrupt: 

In [28]:
input_mat = torch.eye(abundance.shape[1]).to(device)
rel = np.abs(model(input_mat).detach().cpu().numpy())
calc_nondiag_score(rel, adj, metrics=[average_precision_score, roc_auc_score])

[0.29848434836800364, 0.5378928571428572]

In [68]:
model = Abs_LV(10).to('cuda')
loss_fn = torch.nn.MSELoss().to('cuda')
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10000, gamma=0.9)

In [69]:
epoch_num = 300000
for epoch in range(epoch_num):
    input_data = torch.from_numpy(z[:-1, :]).float().to('cuda')
    target_data = torch.from_numpy(z[1:, :]).float().to('cuda')
    # print(input_data.shape, target_data.shape)
    rel = model(input_data)
    loss = loss_fn(rel, target_data)

    sparse_loss = torch.norm(model.interaction, p=1) + torch.norm(model.g, p=1)
    loss += sparse_loss * 1e-10
    
    print(f"Epoch: {epoch}, Loss: {loss.item()}", end='\x1b[1K\r')
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()

In [20]:
a = model.interaction.data.cpu().numpy()
g = model.g.data.cpu().numpy()

In [21]:
calc_nondiag_score(np.abs(a), adj, metrics=[average_precision_score, roc_auc_score])

[0.8813455988455987, 0.92]

In [60]:
calc_nondiag_score(np.abs(a), adj, metrics=[average_precision_score, roc_auc_score])

[0.38180784990377203, 0.6128571428571429]

In [18]:
# %%capture
prauc = []
rocauc = []
for i in range(1):
    z, x, y, adj, M = simulate_glv(
            num_taxa=5,
            # avg_degree=np.random.randint(5, 20),
            avg_degree=2,
            time_points=1000,
            time_step=0.01,
            downsample=1,
            noise_var=0,
            max_interaction_strength=1,
            seed=datetime.now().microsecond,

        )
    model = Abs_LV(5).to('cuda')
    loss_fn = torch.nn.MSELoss().to('cuda')
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-1)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10000, gamma=0.9)
    epoch_num = 300000
    for epoch in range(epoch_num):
        input_data = torch.from_numpy(z[:-1, :]).float().to('cuda')
        target_data = torch.from_numpy(z[1:, :]).float().to('cuda')
        # print(input_data.shape, target_data.shape)
        output = model(input_data)
        loss = loss_fn(output, target_data)

        sparse_loss = torch.norm(model.interaction, p=1) + torch.norm(model.g, p=1)
        # sparse_loss = torch.norm(model.interaction, p=1)
        loss += sparse_loss * 1e-10
        
        print(f"Epoch: {epoch}, Loss: {loss.item()}", end='\x1b[2K\r')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
    a = model.interaction.data.cpu().numpy()
    # g = model.g.data.cpu().numpy()
    pr, roc = calc_nondiag_score(np.abs(a), adj, metrics=[average_precision_score, roc_auc_score])
    prauc.append(pr)
    rocauc.append(roc)


Node 'mgx' initialized
Interaction 'mgx->mgx' added
set m:(mgx)->(mgx):   0:5    0:5
Added x0 vector to node mgx
Added growth rates to node mgx
Initialized
Interaction 'mgx_mgx' added


KeyboardInterrupt: 

In [19]:
print(np.mean(prauc))
print(np.mean(rocauc))

nan
nan


c:\Users\dorasir\anaconda3\envs\net-simu\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\dorasir\anaconda3\envs\net-simu\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [85]:
np.savetxt('../data/results/abslv_auprc.txt', prauc)
np.savetxt('../data/results/abslv_roc.txt', rocauc)

In [ ]:
# %%capture
prauc = []
rocauc = []
for i in range(1):
    z, x, y, adj, M = simulate_glv(
                num_taxa=5,
                # avg_degree=np.random.randint(5, 20),
                avg_degree=2,
                time_points=1000,
                time_step=0.01,
                downsample=1,
                noise_var=0,
                max_interaction_strength=1,
            )
    model = Rel_LV(5).to('cuda')
    # loss_fn = torch.nn.MSELoss().to('cuda')
    loss_fn = torch.nn.L1Loss().to('cuda')
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10000, gamma=0.9)
    epoch_num = 300000
    for epoch in range(epoch_num):
        input_data = torch.from_numpy(x[:-1, :]).float().to('cuda') * 10000
        target_data = torch.from_numpy(x[1:, :]).float().to('cuda')
        # print(input_data.shape, target_data.shape)
        rel = model(input_data, steps=10)
        loss = loss_fn(rel, target_data)

        # sparse_loss = torch.norm(model.interaction, p=1) + torch.norm(model.g, p=1)
        # loss += sparse_loss * 1e-10
        
        print(f"Epoch: {epoch}, Loss: {loss.item()}", end='\x1b[2K\r')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

    a = model.interaction.data.cpu().numpy()
    g = model.g.data.cpu().numpy()
    pr, roc = calc_nondiag_score(np.abs(a), adj, metrics=[average_precision_score, roc_auc_score])
    prauc.append(pr)
    rocauc.append(roc)

In [3]:
print(np.mean(prauc))
print(np.mean(rocauc))
# np.savetxt('../data/results/rellv_prauc.txt', prauc)
# np.savetxt('../data/results/rellv_roc.txt', rocauc)

0.5071734852694605
0.32000000000000006


In [13]:
# %%capture
prauc = []
rocauc = []
device = 'cpu'
for i in range(1):
    z, x, y, adj, M = simulate_glv(
                num_taxa=5,
                # avg_degree=np.random.randint(5, 20),
                avg_degree=2,
                time_points=1000,
                time_step=0.01,
                downsample=1,
                noise_var=0,
                max_interaction_strength=1,
                seed=datetime.now().microsecond,
            )
    model = Compo_LV(5).to(device)
    loss_fn = torch.nn.MSELoss().to(device)
    # optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
    # optimizer = torch.optim.NAdam(model.parameters(), lr=1e-2)
    # optimizer = torch.optim.RAdam(model.parameters(), lr=1e-2)
    optimizer = torch.optim.SGD(model.parameters(), lr=5e-2)
    # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=500, gamma=0.9)
    # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=500, gamma=0.9)
    epoch_num = 1 + 20000
    input_data = torch.from_numpy(x[0, :]).float().to(device) * 10
    target_data = torch.from_numpy(x[1:, :]).float().to(device)
    for epoch in range(epoch_num):
        # print(input_data.shape, target_data.shape)
        rel = model(input_data, steps=target_data.shape[0])
        # output = model(input_data, steps=9)
        loss = loss_fn(rel, target_data)
        # loss = loss_fn(output[-1,:], target_data[-1,:])

        # sparse_loss = torch.norm(model.interaction, p=1)
        # loss += sparse_loss * 1e-6
        
        sys.stdout.write('\x1b[2K')
        print(f"Epoch: {epoch}, Loss: {loss.item()}", end='\r')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # scheduler.step()
        if epoch % 500 == 0:
            a = model.interaction.data.numpy()
            g = model.g.data.numpy()
            pr, roc = calc_nondiag_score(np.abs(a), adj, metrics=[average_precision_score, roc_auc_score])
            print(f"Epoch: {epoch}, Loss: {loss.item()}, AUPRC: {pr}, AUROC: {roc}")
    if device == 'cuda':
        a = model.interaction.data.cpu().numpy()
        g = model.g.data.cpu().numpy()
    else:
        a = model.interaction.data.numpy()
        g = model.g.data.numpy()
    pr, roc = calc_nondiag_score(np.abs(a), adj, metrics=[average_precision_score, roc_auc_score])
    prauc.append(pr)
    rocauc.append(roc)

Node 'mgx' initialized
Interaction 'mgx->mgx' added
set m:(mgx)->(mgx):   0:5    0:5
Added x0 vector to node mgx
Added growth rates to node mgx
Initialized
Interaction 'mgx_mgx' added
Epoch: 0, Loss: 0.0505216047167778, AUPRC: 0.5854628214922333, AUROC: 0.53
Epoch: 500, Loss: 0.008357464335858822, AUPRC: 0.4830460767302872, AUROC: 0.41
Epoch: 1000, Loss: 0.0018467504996806383, AUPRC: 0.47842000835421883, AUROC: 0.39
Epoch: 1500, Loss: 0.0008301680791191757, AUPRC: 0.4846333783175888, AUROC: 0.39999999999999997
Epoch: 2000, Loss: 0.0006143679493106902, AUPRC: 0.49109391124871, AUROC: 0.42000000000000004
Epoch: 2500, Loss: 0.0005543466540984809, AUPRC: 0.4937254901960784, AUROC: 0.43
Epoch: 3000, Loss: 0.0005325202946551144, AUPRC: 0.4937254901960784, AUROC: 0.43
Epoch: 3500, Loss: 0.0005213477415964007, AUPRC: 0.4937254901960784, AUROC: 0.43
Epoch: 4000, Loss: 0.0005130813806317747, AUPRC: 0.4937254901960784, AUROC: 0.43
Epoch: 4500, Loss: 0.0005054890061728656, AUPRC: 0.493725490196078

In [15]:
# Continue
start_point = epoch + 1
input_data = torch.from_numpy(x[0, :]).float().to(device)
target_data = torch.from_numpy(x[1:, :]).float().to(device)
for epoch in range(start_point, start_point + 50000):
        # print(input_data.shape, target_data.shape)
        rel = model(input_data, steps=target_data.shape[0])
        # output = model(input_data, steps=9)
        # loss = loss_fn(output, target_data)
        loss = loss_fn(rel[-1,:], target_data[-1,:])

        # sparse_loss = torch.norm(model.interaction, p=1)
        # loss += sparse_loss * 1e-6
        
        sys.stdout.write('\x1b[2K')
        print(f"Epoch: {epoch}, Loss: {loss.item()}", end='\r')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # scheduler.step()
        if epoch % 500 == 0:
            a = model.interaction.data.numpy()
            g = model.g.data.numpy()
            pr, roc = calc_nondiag_score(np.abs(a), adj, metrics=[average_precision_score, roc_auc_score])
            print(f"Epoch: {epoch}, Loss: {loss.item()}, AUPRC: {pr}, AUROC: {roc}")

Epoch: 61000, Loss: 3.801764250965789e-06, AUPRC: 0.49977832624891444, AUROC: 0.42
Epoch: 61500, Loss: 3.6986893974244595e-06, AUPRC: 0.5036244800950683, AUROC: 0.42999999999999994
Epoch: 62000, Loss: 3.5984746773465304e-06, AUPRC: 0.5036244800950683, AUROC: 0.42999999999999994
Epoch: 62500, Loss: 3.5009707062272355e-06, AUPRC: 0.5036244800950683, AUROC: 0.42999999999999994
Epoch: 63000, Loss: 3.406072892175871e-06, AUPRC: 0.4991800356506238, AUROC: 0.42
Epoch: 63500, Loss: 3.313736215204699e-06, AUPRC: 0.4958467023172905, AUROC: 0.4099999999999999
Epoch: 64000, Loss: 3.223942940167035e-06, AUPRC: 0.4991800356506238, AUROC: 0.42
Epoch: 64500, Loss: 3.136662371616694e-06, AUPRC: 0.4991800356506238, AUROC: 0.42
Epoch: 65000, Loss: 3.0517217055603396e-06, AUPRC: 0.4991800356506238, AUROC: 0.42
Epoch: 65500, Loss: 2.969084334836225e-06, AUPRC: 0.4991800356506238, AUROC: 0.42
Epoch: 66000, Loss: 2.88864748654305e-06, AUPRC: 0.4991800356506238, AUROC: 0.42
Epoch: 66500, Loss: 2.8103663680667

In [38]:
f = open('D:\microbial_network\microbial_network_explore\data\output.txt', 'w')
prauc = []
rocauc = []
device = 'cpu'
for i in range(1):
    z, x, y, adj, M = simulate_glv(
                num_taxa=5,
                avg_degree=3,
                time_points=1000,
                time_step=0.01,
                downsample=1,
                noise_var=0,
                max_interaction_strength=1,
                seed=datetime.now().microsecond,
            )
    model = Compo_LV(5).to(device)
    loss_fn = torch.nn.MSELoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=1e-6)
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=1000, eta_min=1e-6)
    # epoch_num = 1 + 20000
    epoch_num = 1 + 300000
    # input_data = torch.from_numpy(x[0, :]).float().to(device)
    # target_data = torch.from_numpy(x[1:, :]).float().to(device)
    x = torch.from_numpy(x).float().to(device)
    for epoch in range(epoch_num):
        input_data = x[0, :] * 10
        target_data = x[1, :]
        abs, rel = model(input_data)
        # print(abs.sum())
        loss = loss_fn(rel, target_data)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # break
        for i in range(1, x.shape[0]-1):
            abs = abs.detach()
            # while abs.sum() > 10:
            #     abs = abs / 10
            # print(abs.sum())
            input_data = abs.sum() * x[i, :]
            target_data = x[i+1, :]
            abs, rel = model(input_data)
            loss = loss_fn(rel, target_data)
            # sparse_loss = torch.norm(model.interaction, p=1)
            # loss += sparse_loss * 1e-5  
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # sparse_loss = torch.norm(model.interaction, p=1)
        # loss += sparse_loss * 1e-6
        
        # sys.stdout.write('\x1b[2K')
        print(" " * 100, end='\r')
        print(f"Epoch: {epoch}, Loss: {loss.item()}", end='\r')
        
        # scheduler.step()
        if epoch % 100 == 0:
            a = model.interaction.data.numpy()
            g = model.g.data.numpy()
            pr, roc = calc_nondiag_score(np.abs(a), adj, metrics=[average_precision_score, roc_auc_score])
            # print(f"Epoch: {epoch}, Loss: {loss.item()}, AUPRC: {pr}, AUROC: {roc}", file=f, flush=True)
            print(f"Epoch: {epoch}, Loss: {loss.item()}, AUPRC: {pr}, AUROC: {roc}")
    if device == 'cuda':
        a = model.interaction.data.cpu().numpy()
        g = model.g.data.cpu().numpy()
    else:
        a = model.interaction.data.numpy()
        g = model.g.data.numpy()
    pr, roc = calc_nondiag_score(np.abs(a), adj, metrics=[average_precision_score, roc_auc_score])
    prauc.append(pr)
    rocauc.append(roc)

Node 'mgx' initialized
Interaction 'mgx->mgx' added
set m:(mgx)->(mgx):   0:5    0:5
Added x0 vector to node mgx
Added growth rates to node mgx
Initialized
Interaction 'mgx_mgx' added


c:\Users\dorasir\anaconda3\envs\net-simu\lib\site-packages\misdeed-1.0.2-py3.11.egg\misdeed\OmicsGenerator.py:802: RuntimeWarning: overflow encountered in multiply
c:\Users\dorasir\anaconda3\envs\net-simu\lib\site-packages\scipy\integrate\_ivp\common.py:112: RuntimeWarning: invalid value encountered in multiply
  y1 = y0 + h0 * direction * f0
c:\Users\dorasir\anaconda3\envs\net-simu\lib\site-packages\scipy\integrate\_ivp\common.py:114: RuntimeWarning: invalid value encountered in subtract
  d2 = norm((f1 - f0) / scale) / h0


Epoch: 0, Loss: nan, AUPRC: 0.5196428571428572, AUROC: 0.56                                         


KeyboardInterrupt: 

In [11]:
# %%capture
def clr_transform(data):
    # Calculate the geometric mean of each row
    gm = np.exp(np.mean(np.log(data), axis=1, keepdims=True))

    # Calculate the CLR-transformed values
    # clr = np.log(data / gm)
    clr = data / gm

    return clr
prauc = []
rocauc = []
for i in range(1):
    z, x, y, adj, M = simulate_glv(
            num_taxa=5,
            # avg_degree=np.random.randint(5, 20),
            avg_degree=2,
            time_points=1000,
            time_step=0.01,
            downsample=1,
            noise_var=0,
            max_interaction_strength=1,
            seed=datetime.now().microsecond,

        )
    model = Abs_LV(5).to('cuda')
    loss_fn = torch.nn.MSELoss().to('cuda')
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-1)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10000, gamma=0.9)
    epoch_num = 300000
    z = clr_transform(x)
    for epoch in range(epoch_num):
        input_data = torch.from_numpy(z[:-1, :]).float().to('cuda')
        target_data = torch.from_numpy(z[1:, :]).float().to('cuda')
        # print(input_data.shape, target_data.shape)
        output = model(input_data)
        loss = loss_fn(output, target_data)

        sparse_loss = torch.norm(model.interaction, p=1) + torch.norm(model.g, p=1)
        # sparse_loss = torch.norm(model.interaction, p=1)
        loss += sparse_loss * 1e-10
        
        print(f"Epoch: {epoch}, Loss: {loss.item()}", end='\x1b[2K\r')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        if epoch % 5000 == 0:
            a = model.interaction.data.cpu().numpy()
            g = model.g.data.cpu().numpy()
            pr, roc = calc_nondiag_score(np.abs(a), adj, metrics=[average_precision_score, roc_auc_score])
            # print(f"Epoch: {epoch}, Loss: {loss.item()}, AUPRC: {pr}, AUROC: {roc}", file=f, flush=True)
            print(f"Epoch: {epoch}, Loss: {loss.item()}, AUPRC: {pr}, AUROC: {roc}")
    a = model.interaction.data.cpu().numpy()
    # g = model.g.data.cpu().numpy()
    pr, roc = calc_nondiag_score(np.abs(a), adj, metrics=[average_precision_score, roc_auc_score])
    prauc.append(pr)
    rocauc.append(roc)


Node 'mgx' initialized
Interaction 'mgx->mgx' added
set m:(mgx)->(mgx):   0:5    0:5
Added x0 vector to node mgx
Added growth rates to node mgx
Initialized
Interaction 'mgx_mgx' added
Epoch: 0, Loss: 768.8861694335938, AUPRC: 0.5603407213701331, AUROC: 0.56
Epoch: 5000, Loss: 5.991274883854203e-05, AUPRC: 0.7152292070674423, AUROC: 0.65
Epoch: 10000, Loss: 0.005935227498412132, AUPRC: 0.6613690476190476, AUROC: 0.6000000000000001
Epoch: 15000, Loss: 1.4007032405061182e-05, AUPRC: 0.708015873015873, AUROC: 0.65
Epoch: 20000, Loss: 0.01743469014763832, AUPRC: 0.7262148962148962, AUROC: 0.72
Epoch: 25000, Loss: 0.0008301378111355007, AUPRC: 0.699594017094017, AUROC: 0.66
Epoch: 30000, Loss: 0.0031353295780718327, AUPRC: 0.6186999550157445, AUROC: 0.5599999999999999
Epoch: 35000, Loss: 4.223721134621883e-06, AUPRC: 0.5667077418625406, AUROC: 0.51
Epoch: 40000, Loss: 2.932983079517726e-06, AUPRC: 0.568489010989011, AUROC: 0.53
Epoch: 45000, Loss: 0.007312569301575422, AUPRC: 0.5964301874595

KeyboardInterrupt: 

In [72]:
print(np.mean(prauc))
print(np.mean(rocauc))

0.5883141994171406
0.32999999999999996


Compare ours and their program